# About this notebook

#### Implemented by Rodrigo Costa, 2022.
#### 
#### Download all files in the repository and save them in the same folder.
#### Change the pathing in the first code cell in this notebook to reflect the location of this folder.

#### Inputs included were obtained from:
#### https://www.fema.gov/openfema-data-page/individual-assistance-housing-registrants-large-disasters-v1



---



---



In [ ]:
# Mount YOUR google drive. You'll need to "Add shortcut to Drive" for our shared folder for it to show up here.
# Use the URL shown below in the output to authorize this Colab session to access you GDrive
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

# Modify this according to the path in your computer
data_dir = '/content/drive/MyDrive/SURI/Studies/2022_Wildfires/' # <-- change this to reflect the pathing in your machine

Mounted at /content/drive/


In [ ]:
# Import needed packages
! pip install geopandas
! pip install geopy
! pip install -U plotly
! pip install cmcrameri
! pip install cpi
! pip install requests
!pip install adjustText

import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import geopy.distance
from shapely.geometry import Point, Polygon
import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy 
from scipy import stats as sts
import plotly.express as px
from numpy.random import default_rng
from plotnine import *
import time
from cmcrameri import cm
import cpi
import requests 

rng = default_rng(13)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 7.3 MB/s 
     |████████████████████████████████| 6.3 MB 59.4 MB/s 
     |████████████████████████████████| 16.7 MB 54.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 8.3 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 134 kB 8.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.6 MB 1.2 MB/s 
Lookin

# Approval rates

## Get hold of the data and clean it

In [ ]:
FEMA_dir = data_dir + 'FEMA_IAP/'


# Uncomment this if you want to re-read the whole/large file
df_FEMA = pd.read_csv(FEMA_dir + "IndividualAssistanceHousingRegistrantsLargeDisasters_CA.csv")
df_FEMA = df_FEMA[['disasterNumber','damagedCity','damagedStateAbbreviation','censusBlockId','householdComposition','ownRent','residenceType','homeOwnersInsurance','grossIncome',\
                    'primaryResidence','ppfvl','rpfvl','rentalAssistanceAmount','repairAmount','replacementAmount']]


# Drop anyone without Real Property FEMA Verified Losses because they are automatically denied assistance
df_FEMA[pd.to_numeric(df_FEMA['rpfvl'], errors='coerce').notnull()]
df_FEMA = df_FEMA[df_FEMA['rpfvl'].isna() == False]
df_FEMA = df_FEMA[df_FEMA['rpfvl'].astype(int) > 0].reset_index(drop=True)


# Remove data not available in 2017
df_FEMA_IHP = df_FEMA[~df_FEMA['disasterNumber'].isin(['4344','4353'])].reset_index(drop=True)

# Make a copy of the original data
df_FEMA_IHP = df_FEMA.copy()


# Remove 'NaN' income
df_FEMA_IHP['grossIncome'] = df_FEMA_IHP['grossIncome'].fillna(-1)
#df_FEMA_IHP = df_FEMA_IHP[~df_FEMA_IHP['grossIncome'].isin(['NaN','nan'])].reset_index(drop=True)


# Remove 'NaN' repair/repair amount
#df_FEMA_IHP = df_FEMA_IHP[df_FEMA_IHP['repairAmount'].notna()].reset_index(drop=True)
df_FEMA_IHP['repairAmount'] = df_FEMA_IHP['repairAmount'].fillna(0)
df_FEMA_IHP['replacementAmount'] = df_FEMA_IHP['replacementAmount'].fillna(0)
df_FEMA_IHP['totalAmount'] = df_FEMA_IHP['repairAmount'] + df_FEMA_IHP['replacementAmount']


# Clean loss data 'NaN' loss
df_FEMA_IHP = df_FEMA_IHP[df_FEMA_IHP['rpfvl'].notna()].reset_index(drop=True)
df_FEMA_IHP['rpfvl'] = df_FEMA_IHP['rpfvl'].apply(lambda x: 1000000 if x > 1000000 else x)


# Remove mobile homes
df_FEMA_IHP = df_FEMA_IHP[df_FEMA_IHP['residenceType'] != 'Mobile Home'].reset_index(drop=True)


# Remove data for Puerto Rico as losses/incomes are very different from continental US
df_FEMA_IHP = df_FEMA_IHP[df_FEMA_IHP['damagedStateAbbreviation'] != 'PR'].reset_index(drop=True)


# Disaster year
df_FEMA_Declarations = pd.read_csv(data_dir + "DisasterDeclarationsSummaries.csv")
df_FEMA_Declarations = df_FEMA_Declarations[df_FEMA_Declarations['fyDeclared']>2000]
df_FEMA_Declarations = df_FEMA_Declarations[df_FEMA_Declarations['declarationType'] == 'DR']
codes = df_FEMA_Declarations['disasterNumber']
years = df_FEMA_Declarations['fyDeclared']
FEMADeclaration_dict = dict(zip(codes, years))
df_FEMA_IHP['Year'] = df_FEMA_IHP['disasterNumber'].map(FEMADeclaration_dict)


# Grant to loss ratio
df_FEMA_IHP['Ratio'] = df_FEMA_IHP['totalAmount'] / df_FEMA_IHP['rpfvl']
df_FEMA_IHP['Ratio'] = df_FEMA_IHP['Ratio'].apply(lambda x: 1 if x > 1 else x)


# Income and loss in 2022 dollars
df_FEMA_IHP['grossIncome'] = df_FEMA_IHP['grossIncome'] #* df_FEMA_IHP['InflationRates']
df_FEMA_IHP['totalAmount'] = df_FEMA_IHP['totalAmount'] #* df_FEMA_IHP['InflationRates']
df_FEMA_IHP['rentalAssistanceAmount'] = df_FEMA_IHP['rentalAssistanceAmount'] #* df_FEMA_IHP['InflationRates']


# Disaster code
dic_disaster = {4332: 'Harvey',
                4337: 'Irma',
                4339: 'Maria',
                4559: 'Laura',
                4611: 'Ida',
                4399: 'Michael',
                4586: 'Texas Winter Storms',
                4393: 'Florence'}
#df_FEMA_IHP = df_FEMA_IHP.replace({"disasterNumber": dic_disaster})

                
# Income brackets
def getIncomeBracket(row):
    # Median household income per state
    dic_income = {'TX': 61874,
                  'PR': 19775,
                  'FL': 55660,
                  'LA': 49469,
                  'NC': 54602}


    ami = dic_income[row['damagedStateAbbreviation']]
    inc_bracket = 'None'
    if row['grossIncome'] < 0:
        inc_bracket = 'Unknown'
    elif row['grossIncome'] < 0.5*ami:
        inc_bracket = 'VeryLow'
    elif row['grossIncome'] < 0.8*ami:
        inc_bracket = 'Low'
    elif row['grossIncome'] < 1.2*ami:
        inc_bracket = 'Moderate'
    else: 
        inc_bracket = 'High'
    return inc_bracket


# Insurance
df_FEMA_IHP['homeOwnersInsurance'] = df_FEMA_IHP['homeOwnersInsurance'].apply(lambda x: 'Insured' if x == 1 else 'Uninsured')


# Residence type
df_FEMA_IHP['residenceType'] = df_FEMA_IHP['residenceType'].apply(lambda x: x if x in ['House/Duplex','Condo','Townhouse'] else 'Other')


df_FEMA_IHP['IncomeBracket'] = df_FEMA_IHP.apply(lambda row: getIncomeBracket(row), axis=1)
df_FEMA_IHP = df_FEMA_IHP[df_FEMA_IHP['IncomeBracket'] != 'Unknown'].reset_index(drop=True)

## Approval rates

In [ ]:
# Filter data
def filterIHPData(df,income,residenceType,insurance,loss,state,what):

    df_bkp = df.copy()

    # filter by income
    if income in df['IncomeBracket'].unique():
        df = df[df['IncomeBracket'] == income]

    # filter by residenceType
    if residenceType in df['residenceType'].unique():
        df = df[df['residenceType'] == residenceType]

    # filter by state
    if state in df['damagedStateAbbreviation'].unique():
        df = df[df['damagedStateAbbreviation'] == state]

    # filter by insurance
    if insurance in df['homeOwnersInsurance'].unique():
        df = df[df['homeOwnersInsurance'] == insurance]

    # filter by rpfvl
    df = df[(df['rpfvl'] >= loss-10000) & (df['rpfvl'] < loss)]

    if len(df.index) < 100:
        print('Warning: small dataset for this combination of income, residenceType,state, and insurance status!',income,residenceType,state,insurance)
        df = df_bkp

    if what == 'amount':
        # approval amount
        return df[df['totalAmount']>0]['totalAmount'].mean()
    
    elif what == 'approvalrate':
        return len(df[df['totalAmount']>0].index)/len(df.index)

# Approval rate
def getFEMAApprovalRateMatrix(state):

    # Calculate the approval rates once
    df_ApprovalRates = pd.DataFrame()

    # Loss brackets
    loss_brackets = [10000,20000,30000,40000]

    income = []
    housing = []
    insurance = []
    loss = []
    values = []
    for i in df_FEMA_IHP['IncomeBracket'].unique():
        for j in df_FEMA_IHP['residenceType'].unique():
            for k in df_FEMA_IHP['homeOwnersInsurance'].unique():
                for l in loss_brackets:
                    income.append(i)
                    housing.append(j)
                    insurance.append(k)
                    loss.append(l)
                    values.append(filterIHPData(df_FEMA_IHP,i,j,k,l,state,'approvalrate'))
                    #print(str(i)+'_'+str(j)+'_'+str(k),getApprovalRate(df_FEMA_Selected,i,j,'CA',k))

    df_ApprovalRates['Income'] = income
    df_ApprovalRates['Residence Type'] = housing
    df_ApprovalRates['Insurance Status'] = insurance
    df_ApprovalRates['Loss'] = loss
    df_ApprovalRates['Approval Rates'] = values
    df_ApprovalRates.to_csv(data_dir+ 'FEMA_IAP/' + 'df_ApprovalRates.txt')

    return df_ApprovalRates


# Select the approval rate based on demographics
def getIHPApprovalRate(df,income,residence,insurance,loss):

    loss_cat = 10000 if loss < 10000 else (20000 if loss < 20000 else (30000 if loss < 30000 else 40000))

    idx = np.where((df['Income'] == income) &\
              (df['Residence Type'] == residence) &\
              (df['Insurance Status'] == insurance) &\
              (df['Loss'] == loss_cat))

    return float(df.loc[idx,'Approval Rates'])

## <font color='blue'> Use the cell below to generate the approval rate matrix for one state</font>

In [ ]:
theState = 'CA'
df_FEMAIHPAppovalRates = getFEMAApprovalRateMatrix(theState)
df_FEMAIHPAppovalRates

,Income,Residence Type,Insurance Status,Loss,Approval Rates
0,Moderate,House/Duplex,Uninsured,10000,0.485075
1,Moderate,House/Duplex,Uninsured,20000,0.786706
2,Moderate,House/Duplex,Uninsured,30000,0.746556
3,Moderate,House/Duplex,Uninsured,40000,0.843434
4,Moderate,House/Duplex,Insured,10000,0.223487
...,...,...,...,...,...
123,High,Townhouse,Uninsured,40000,0.383402
124,High,Townhouse,Insured,10000,0.205993
125,High,Townhouse,Insured,20000,0.375000
126,High,Townhouse,Insured,30000,0.383402


## Approved amount


In [ ]:
# Approval amount
def getFEMAApprovedAmountMatrix(state):

    # Calculate the approval rates once
    df_ApprovalAmount = pd.DataFrame()

    # Loss brackets
    loss_brackets = [10000,20000,30000,40000]

    income = []
    housing = []
    insurance = []
    loss = []
    values = []
    for i in df_FEMA_IHP['IncomeBracket'].unique():
        for j in df_FEMA_IHP['residenceType'].unique():
            for k in df_FEMA_IHP['homeOwnersInsurance'].unique():
                for l in loss_brackets:
                    income.append(i)
                    housing.append(j)
                    insurance.append(k)
                    loss.append(l)
                    values.append(filterIHPData(df_FEMA_IHP,i,j,k,l,state,'amount'))
                    #print(str(i)+'_'+str(j)+'_'+str(k),getApprovalRate(df_FEMA_Selected,i,j,'CA',k))

    df_ApprovalAmount['Income'] = income
    df_ApprovalAmount['Residence Type'] = housing
    df_ApprovalAmount['Insurance Status'] = insurance
    df_ApprovalAmount['Loss'] = loss
    df_ApprovalAmount['Approved Amount'] = values
    df_ApprovalAmount.to_csv(data_dir+ 'FEMA_IAP/' + 'df_ApprovedAmount.txt')

    return df_ApprovalAmount


# Select the approval rate based on demographics
def getIHPApprovedAmount(df,income,residence,insurance,loss):
    loss_cat = 10000 if loss < 10000 else\
                (20000 if loss < 20000 else\
                 (30000 if loss < 30000 else 40000))
    
    idx = np.where((df['Income'] == income) &\
              (df['Residence Type'] == residence) &\
              (df['Insurance Status'] == insurance) &\
              (df['Loss'] == loss_cat))

    return float(df.loc[idx,'Approved Amount'])

## <font color='blue'> Use the cell below to generate the approval amount matrix for one state</font>

In [ ]:
df_FEMAIHPApprovedAmounts = getFEMAApprovedAmountMatrix('CA')
df_FEMAIHPApprovedAmounts

,Income,Residence Type,Insurance Status,Loss,Approved Amount
0,Moderate,House/Duplex,Uninsured,10000,3728.076432
1,Moderate,House/Duplex,Uninsured,20000,14145.575574
2,Moderate,House/Duplex,Uninsured,30000,22231.027897
3,Moderate,House/Duplex,Uninsured,40000,30054.891976
4,Moderate,House/Duplex,Insured,10000,3518.973719
...,...,...,...,...,...
123,High,Townhouse,Uninsured,40000,7174.445992
124,High,Townhouse,Insured,10000,4459.199333
125,High,Townhouse,Insured,20000,13199.047738
126,High,Townhouse,Insured,30000,7174.445992


## Calculate grant 

#### <font color='red'> The eligible loss should be: loss - insurance payments </font>

In [ ]:
def getFEMAGrant(loss,insured,income,residenceType,state):

    # Maximum grant
    fema_max = 34000 # inflation
    returnApproval = [0] 
    returnAmount = [0]

    # Eligible loss
    theEligibleLoss = loss # Any amount from insurance should be subtracted here!!!

    # Approval rate
    fema_approval = getIHPApprovalRate(df_FEMAIHPAppovalRates,income,residenceType,insured,loss)
    p = [1-fema_approval,fema_approval]
    s = [0,1]

    # Amount approved
    fema_amount = min(fema_max,getIHPApprovedAmount(df_FEMAIHPApprovedAmounts,income,residenceType,insured,loss))

    returnApproval = fema_approval

    returnAmount = min(fema_amount,theEligibleLoss) 

    return returnApproval,returnAmount

## <font color='blue'> Use the cell below to calculate the approval rate and amount for a family with known demographics </font>

In [ ]:
theState = 'CA' # California
theIncome = 'Low' # Very Low, Low, Moderate, or High
theResidenceType = 'House/Duplex' # House/Duplex, Other, Condo, Townhouse
theInsuranceStatus = 'Uninsured' # Insured, Uninsured
theLoss = 65000 # real value. Note: this should be the losses deducted of the insurance amount received (if any)

print('The FEMA IAP approval rate for a family whose income is', theIncome, ' \nin a', theResidenceType, ' \nthat is', theInsuranceStatus, ' \nand experienced a loss of', theLoss, ' \nis',\
getFEMAGrant(theLoss,theInsuranceStatus,theIncome,theResidenceType,theState)[0])
print('\n')
print('If approved, a family whose income is', theIncome, ' \nin a', theResidenceType, ' \nthat is', theInsuranceStatus, ' \nand eligible losses equal to', theLoss, ' \nis expected to receive',\
getFEMAGrant(theLoss,theInsuranceStatus,theIncome,theResidenceType,theState)[1])

The FEMA IAP approval rate for a family whose income is Low  
in a House/Duplex  
that is Uninsured  
and experienced a loss of 65000  
is 0.8082706766917294


If approved, a family whose income is Low  
in a House/Duplex  
that is Uninsured  
and eligible losses equal to 65000  
is expected to receive 30510.827023255817


#### <font color='red'> Note: </font> depending on the combination of state and hazard you select, there might be very few data points available in the data and the results should be taken with a (or many) grain(s) of salt (see warning above).